# Calculation of SVERAD SVs (for SVM) and SHAP values (for RF and SVM)

This notebook will generate:

* Exact SVs for SVM with SVERAD
* SHAP values for SVM with KernelSHAP
* (Exact?) SVs for RF with TreeExplainer
* SHAP values for RF with KernelSHAP

In [1]:
from src.sverad_svm import ExplainingSVC, create_SVC
from src.sverad_kernel import rbf_kernel_matrix_sparse
from src.utils import DataSet, UnfoldedMorganFingerprint, set_seeds


import numpy as np
import pandas as pd
import pickle
import shap
from sklearn import ensemble
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from tqdm.auto import tqdm
import warnings

In [2]:
SEED = 42
set_seeds(SEED)

SAVE_DATASET_PICKLE = False
LOAD_DATASET_PICKLE = True
LOAD_PRECOMPUTED_EXPLANATIONS = False
# SAVE_EXPLANATIONS = True

## Definition of models and searched hyperparameter space.

In [3]:
model_list = [{"name": "SVC",
               "algorithm": ExplainingSVC(),
               "parameter": {"C": [0.1, 1, 10, 50, 100, 200, 400, 500, 750, 1000], 
                            "gamma_value": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], #[0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
                            },
              },
              {"name": "RF",
               "algorithm": ensemble.RandomForestClassifier(random_state=SEED, bootstrap=True, max_features="sqrt"),
               "parameter": {'n_estimators':[10, 100, 250, 500],
                             'min_samples_split': [2, 3, 5, 7, 10,],
                             'min_samples_leaf':[1, 2, 5, 10],
                            },
              },
             ]

## Loading the pre-compiled compound set.

In [4]:
dataset_df = pd.read_csv("./data/dataset.tsv", sep="\t")
dataset_df.pivot_table(index="uniprot_id", columns="label", values="nonstereo_aromatic_smiles", aggfunc="nunique", fill_value=0)

label,active,random
uniprot_id,,
P0DMS8,287,287


## Creating the dataset

In [5]:
dataset_dict = dict()
fingerprint_gen_dict = dict()

if not LOAD_DATASET_PICKLE:
    for dataset_name, data_grpd_df in dataset_df.groupby("uniprot_id"):
        
        # label: 1: active, 0: random
        labels = np.array([1 if l == 'active' else 0 for l in data_grpd_df.label])
        # Creating Fingerprint
        morgan_radius2 = UnfoldedMorganFingerprint(radius=2)
        morgan_radius2.fit_smiles(data_grpd_df.nonstereo_aromatic_smiles.tolist())
        
        # Constructing Dataset
        fp_matrix = morgan_radius2.transform_smiles(data_grpd_df.nonstereo_aromatic_smiles.tolist())
        # Constructing Dataset
        dataset = DataSet(labels, fp_matrix)
        dataset.add_attribute("nonstereo_aromatic_smiles", data_grpd_df.nonstereo_aromatic_smiles.values)
        
        dataset_dict[dataset_name] = dataset
        fingerprint_gen_dict[dataset_name] = morgan_radius2
else:
    with open("./data/pickle/dataset_dict.p", "rb") as infile:
        dataset_dict = pickle.load(infile)
    with open("./data/pickle/fingerprint_gen_dict.p", "rb") as infile:
        fingerprint_gen_dict = pickle.load(infile)

In [6]:
if SAVE_DATASET_PICKLE:
    with open("./pickle_dumps/dataset_dict.pkl", "wb") as outfile:
        pickle.dump(dataset_dict, outfile)
    with open("./pickle_dumps/fingerprint_gen_dict.pkl", "wb") as outfile:
        pickle.dump(fingerprint_gen_dict, outfile)

## Training the models and generation of feature contributions

In [7]:
n_splits = 1 # Number of test-training splits

In [8]:
prediction_df = []
failed = 0
hyperparamter_dict = dict()
obtained_models = dict()
shap_dict = dict()

# Loop over multiple data-sets. Here only one is assessed.
for dataset_name, dataset in tqdm(dataset_dict.items(), total=len(dataset_dict)): #it takes ~5:30h to run Kernel SHAP for SVC + ~7h for RF

    # Loop over test-training splits. Only one is assessed. `n_splits == 1`
    print("Splitting dataset.")
    data_splitter = StratifiedShuffleSplit(n_splits=n_splits, random_state=SEED, test_size=0.50)
    for trial_nr, (train_idx, test_idx) in tqdm(enumerate(data_splitter.split(dataset.feature_matrix, dataset.label)), leave=False, total=n_splits, disable=True):
        training_set = dataset[train_idx]
        test_set = dataset[test_idx]

        #Iterating over assessed models.
        for model_dict in model_list:
            print(model_dict["parameter"])
            # Setting up hyperparameter search.
            param_grid = model_dict["parameter"]
            model = GridSearchCV(estimator = model_dict["algorithm"],
                                 param_grid = param_grid,
                                 n_jobs=1,
                                 scoring= "neg_mean_squared_error", #"neg_mean_squared_error", "accuracy"
                                 cv=StratifiedShuffleSplit(n_splits=10, random_state=SEED, test_size=0.5),
                                verbose=0,
                                )
            # Determining optimal hyperparameters and fitting the model to the entire training set with these hyperparamters
            print("Model fitting and tuning to obtain optimal hyperparameters via Grid Search...")
            model.fit(training_set.feature_matrix, training_set.label)
            preds = model.predict(test_set.feature_matrix)
            print("Model accuracy: ", np.mean(preds == test_set.label))
            obtained_models[(dataset_name, trial_nr, model_dict["name"])] = model
            print("Model trained and optimized.")
            # Saving hyperparameters
            if model_dict["name"] not in hyperparamter_dict:
                hyperparamter_dict[model_dict["name"]] = []
            best_param = dict(model.best_params_)
            best_param["dataset_name"] = dataset_name
            best_param["trial"] = trial_nr
            hyperparamter_dict[model_dict["name"]].append(best_param)

            # break
            # SVs.
            shap_values = None
            print("Model name: ", model_dict["name"])
            if model_dict["name"] == "RF":  # Random forest
                pass #skip RF explanations since we already have those
                # model_explainer = shap.TreeExplainer(model.best_estimator_, feature_perturbation="interventional", data=training_set.feature_matrix.toarray())
                # try:
                #     # first line to sort out cases which fail on the spot
                #     _ = model_explainer.shap_values(test_set.feature_matrix.toarray()[0:2, :], check_additivity=True)  # check_additivity is True in defalult. This is just a reminder why I'm doing this.
                #     print("Explaining model using SHAP TreeExplainer...")
                #     shap_values = model_explainer.shap_values(dataset.feature_matrix.toarray())[1]
                #     expected_value = model_explainer.expected_value[1]
                #     print("SHAP TreeExplainer done.")
                # except Exception as ex:
                #     print(ex)
                #     failed += 1
                #     continue
            elif model_dict["name"] == "SVC":
                print("Explaining model using SVERAD...")
                shap_values = model.best_estimator_.feature_weights(dataset.feature_matrix)
                expected_value = model.best_estimator_.expected_value
                print("SVERAD values computation done.")
            else:
                raise(ValueError("Model not implemented."))
            
            # Kernel SHAP
            shap_sample = shap.sample(training_set.feature_matrix)
            if model_dict["name"] == "RF":
                link = "identity"
            else:
                link = "logit"
            model_explainer = shap.KernelExplainer(model.predict_proba, shap_sample, link=link)
            with warnings.catch_warnings():
                # ignore all caught warnings. Necessary, since LassoLarsIC raises (a lot) future-warnings . See https://github.com/slundberg/shap/issues/2528
                warnings.filterwarnings("ignore")
                print("Explaining model using SHAP KernelExplainer...")
                kernel_shap_values = model_explainer.shap_values(dataset.feature_matrix, nsamples="auto")[1]
                print("SHAP KernelExplainer done.")
            # Creating a DataFrame with all relevant data.
            trial_df = pd.DataFrame()
            trial_df["nonstereo_aromatic_smiles"] = dataset.nonstereo_aromatic_smiles
            trial_df["dataset_idx"] = range(len(dataset.label))
            trial_df["label"] = dataset.label
            trial_df["prediction"] = model.best_estimator_.predict(dataset.feature_matrix)
            if model_dict["name"] == "SVC":
                trial_df["log_odds"] = model.best_estimator_.predict_log_odds(dataset.feature_matrix)[:, 1]
            trial_df["proba"] = model.best_estimator_.predict_proba(dataset.feature_matrix)[:, 1]
            trial_df["trainingset"] = trial_df.nonstereo_aromatic_smiles.isin(training_set.nonstereo_aromatic_smiles)
            trial_df["testset"] = trial_df.nonstereo_aromatic_smiles.isin(test_set.nonstereo_aromatic_smiles)
            trial_df["trial"] = trial_nr
            trial_df["dataset_name"] = dataset_name
            trial_df["algorithm"] = model_dict["name"]
            
            # Creating a data set storing all SVs and SHAP values.
            shap_dict[(dataset_name, trial_nr, model_dict["name"])] = dict()
            if shap_values is not None:
                trial_df["present_shap"] = (shap_values * dataset.feature_matrix.toarray()).sum(axis=1)
                trial_df["absent_shap"] = (shap_values * (1-dataset.feature_matrix.toarray())).sum(axis=1) 
                if model_dict["name"] == "SVC":
                    shap_dict[(dataset_name, trial_nr, model_dict["name"])]["sverad_values"] = shap_values
                else:
                    shap_dict[(dataset_name, trial_nr, model_dict["name"])]["tree_shap_values"] = shap_values #for SVC, this will contain the SVERAD values. Added if statement to deal with that. TODO: CHECK IF CORRECT!
                shap_dict[(dataset_name, trial_nr, model_dict["name"])]["expected_value"] = expected_value
            shap_dict[(dataset_name, trial_nr, model_dict["name"])]["kernel_shap_values"] = kernel_shap_values
            shap_dict[(dataset_name, trial_nr, model_dict["name"])]["kernel_expected_value"] = model_explainer.expected_value[1]
            trial_df["kernel_present_shap"] = (kernel_shap_values * dataset.feature_matrix.toarray()).sum(axis=1)
            trial_df["kernel_absent_shap"] = (kernel_shap_values * (1-dataset.feature_matrix.toarray())).sum(axis=1)
            
            prediction_df.append(trial_df)
# prediction_df = pd.concat(prediction_df)
# print(f"Number of failed datasets: {failed}")

# print("Explanation done. Saving results...")

  0%|          | 0/1 [00:00<?, ?it/s]

Splitting dataset.
{'C': [0.1, 1, 10, 50, 100, 200, 400, 500, 750, 1000], 'gamma_value': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
Model fitting and tuning to obtain optimal hyperparameters via Grid Search...
Model accuracy:  0.9337979094076655
Model trained and optimized.
Model name:  SVC
Explaining model using SVERAD...
SVERAD values computation done.
Explaining model using SHAP KernelExplainer...


  0%|          | 0/574 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print(model.best_params_)
print(model.best_estimator_.get_params())
print(model.best_score_)
model_dict["parameter"]

{'C': 10, 'gamma_value': 0.01}
{'C': 10, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'decision_function_shape': 'ovr', 'gamma_value': 0.01, 'max_iter': -1, 'no_player_value': 0, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
-0.06944444444444445


{'C': [0.1, 1, 10, 50, 100, 200, 400, 500, 750, 1000],
 'gamma_value': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}

In [ ]:
# for g_val in model_dict["parameter"]["gamma_value"]:
#     mod = ExplainingSVC(gamma_value=g_val).fit(training_set.feature_matrix, training_set.label)
#     preds = mod.predict(test_set.feature_matrix)
#     print("Model accuracy: ", np.mean(preds == test_set.label))
#     print("=============================")

Model accuracy:  0.9024390243902439
Model accuracy:  0.9407665505226481
Model accuracy:  0.9337979094076655
Model accuracy:  0.8362369337979094
Model accuracy:  0.49825783972125437
Model accuracy:  0.49825783972125437
Model accuracy:  0.49825783972125437


## Storing all results.

In [ ]:
with open("./data/pickle/shap_dict_only_SVM.p", "wb") as outfile:
    pickle.dump(shap_dict, outfile)
with open("./data/pickle/obtained_models_only_SVM.p", "wb") as outfile:
    pickle.dump(obtained_models, outfile)

In [ ]:
prediction_df.to_csv("./data/prediction_df_SVM_RF.tsv", sep="\t", index=False)

In [ ]:
print("Results saved.")

Results saved.
